**Курсовая работа**

Предсказать цены на квартиры в датасете test.csv. 
Вам будут даны два датасета: train.csv (содержит признаки и цены на квартиры) и test.csv (только признаки).

In [2]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score

In [5]:
X = pd.read_csv('train.csv')

In [6]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             10000 non-null  int64  
 1   DistrictId     10000 non-null  int64  
 2   Rooms          10000 non-null  float64
 3   Square         10000 non-null  float64
 4   LifeSquare     7887 non-null   float64
 5   KitchenSquare  10000 non-null  float64
 6   Floor          10000 non-null  int64  
 7   HouseFloor     10000 non-null  float64
 8   HouseYear      10000 non-null  int64  
 9   Ecology_1      10000 non-null  float64
 10  Ecology_2      10000 non-null  object 
 11  Ecology_3      10000 non-null  object 
 12  Social_1       10000 non-null  int64  
 13  Social_2       10000 non-null  int64  
 14  Social_3       10000 non-null  int64  
 15  Healthcare_1   5202 non-null   float64
 16  Helthcare_2    10000 non-null  int64  
 17  Shops_1        10000 non-null  int64  
 18  Shops_2

In [9]:
X.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Price
count,10000.00000,10000.000000,10000.000000,10000.000000,7887.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,5202.000000,10000.000000,10000.000000,10000.000000
mean,8383.40770,50.400800,1.890500,56.315775,37.199645,6.273300,8.526700,12.609400,3.990166e+03,0.118858,24.687000,5352.157400,8.039200,1142.904460,1.319500,4.231300,214138.857399
std,4859.01902,43.587592,0.839512,21.058732,86.241209,28.560917,5.241148,6.775974,2.005003e+05,0.119025,17.532614,4006.799803,23.831875,1021.517264,1.493601,4.806341,92872.293865
min,0.00000,0.000000,0.000000,1.136859,0.370619,0.000000,1.000000,0.000000,1.910000e+03,0.000000,0.000000,168.000000,0.000000,0.000000,0.000000,0.000000,59174.778028
25%,4169.50000,20.000000,1.000000,41.774881,22.769832,1.000000,4.000000,9.000000,1.974000e+03,0.017647,6.000000,1564.000000,0.000000,350.000000,0.000000,1.000000,153872.633942
50%,8394.50000,36.000000,2.000000,52.513310,32.781260,6.000000,7.000000,13.000000,1.977000e+03,0.075424,25.000000,5285.000000,2.000000,900.000000,1.000000,3.000000,192269.644879
75%,12592.50000,75.000000,2.000000,65.900625,45.128803,9.000000,12.000000,17.000000,2.001000e+03,0.195781,36.000000,7227.000000,5.000000,1548.000000,2.000000,6.000000,249135.462171
max,16798.00000,209.000000,19.000000,641.065193,7480.592129,2014.000000,42.000000,117.000000,2.005201e+07,0.521867,74.000000,19083.000000,141.000000,4849.000000,6.000000,23.000000,633233.466570


In [8]:
X

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2,Price
0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,B,B,33,7976,5,NaN,0,11,B,184966.930730
1,15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,B,B,46,10309,1,240.0,1,16,B,300009.450063
2,4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,B,B,34,7759,0,229.0,1,3,B,220925.908524
3,5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,B,B,23,5735,3,1084.0,0,5,B,175616.227217
4,10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,B,B,35,5776,1,2078.0,2,4,B,150226.531644
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,77,32,2.0,50.401785,30.476203,5.0,6,5.0,1968,0.135650,B,B,46,7960,6,350.0,3,11,B,196684.316040
9996,6159,18,1.0,41.521546,20.539216,9.0,13,13.0,2000,0.000000,B,B,30,5562,0,NaN,0,5,A,189050.289571
9997,5123,27,1.0,47.939008,NaN,1.0,12,16.0,2015,0.072158,B,B,2,629,1,NaN,0,0,A,159143.805370
9998,5400,75,2.0,43.602562,33.840147,8.0,1,5.0,1961,0.307467,B,A,30,5048,9,325.0,2,5,B,181595.339808


In [10]:
y = X['Price']
y

0       184966.930730
1       300009.450063
2       220925.908524
3       175616.227217
4       150226.531644
            ...      
9995    196684.316040
9996    189050.289571
9997    159143.805370
9998    181595.339808
9999    218714.077615
Name: Price, Length: 10000, dtype: float64

In [11]:
X = X.drop('Price', axis = 1)

In [12]:
X

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2
0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,B,B,33,7976,5,NaN,0,11,B
1,15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978,0.000070,B,B,46,10309,1,240.0,1,16,B
2,4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968,0.049637,B,B,34,7759,0,229.0,1,3,B
3,5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977,0.437885,B,B,23,5735,3,1084.0,0,5,B
4,10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976,0.012339,B,B,35,5776,1,2078.0,2,4,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,77,32,2.0,50.401785,30.476203,5.0,6,5.0,1968,0.135650,B,B,46,7960,6,350.0,3,11,B
9996,6159,18,1.0,41.521546,20.539216,9.0,13,13.0,2000,0.000000,B,B,30,5562,0,NaN,0,5,A
9997,5123,27,1.0,47.939008,NaN,1.0,12,16.0,2015,0.072158,B,B,2,629,1,NaN,0,0,A
9998,5400,75,2.0,43.602562,33.840147,8.0,1,5.0,1961,0.307467,B,A,30,5048,9,325.0,2,5,B


In [13]:
X.corr()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1
Id,1.000000,0.012973,-0.005847,-0.010071,0.018449,0.019880,0.001348,-0.008376,0.005004,0.018097,-0.000772,-0.002033,-0.009358,-0.003879,0.001502,-0.008718
DistrictId,0.012973,1.000000,0.071432,-0.026613,-0.019910,0.040358,-0.120373,-0.149051,0.013430,0.065294,0.246463,0.167479,0.136095,0.304335,0.306147,0.174214
Rooms,-0.005847,0.071432,1.000000,0.662893,0.131336,0.005123,-0.000665,-0.029302,-0.010612,-0.032347,0.075980,0.071335,0.012811,0.042857,0.063557,0.053618
Square,-0.010071,-0.026613,0.662893,1.000000,0.196129,0.008320,0.114791,0.081505,-0.009032,-0.064479,-0.070690,-0.043120,0.035241,-0.039748,-0.022960,0.021357
LifeSquare,0.018449,-0.019910,0.131336,0.196129,1.000000,-0.001195,0.024559,0.027442,-0.002210,-0.023629,-0.048363,-0.039283,0.012763,-0.026867,-0.024762,-0.007569
KitchenSquare,0.019880,0.040358,0.005123,0.008320,-0.001195,1.000000,-0.011397,0.000780,0.000958,-0.005622,0.043379,0.037805,-0.015140,0.009472,0.043170,0.010216
Floor,0.001348,-0.120373,-0.000665,0.114791,0.024559,-0.011397,1.000000,0.418986,0.000928,-0.016133,-0.044914,-0.016560,-0.002237,-0.138294,-0.065537,0.024264
HouseFloor,-0.008376,-0.149051,-0.029302,0.081505,0.027442,0.000780,0.418986,1.000000,-0.000864,-0.004362,-0.020801,0.007194,-0.008137,-0.143973,-0.068728,0.026279
HouseYear,0.005004,0.013430,-0.010612,-0.009032,-0.002210,0.000958,0.000928,-0.000864,1.000000,0.001465,0.003026,0.001970,0.000819,-0.011969,0.011245,0.003681
Ecology_1,0.018097,0.065294,-0.032347,-0.064479,-0.023629,-0.005622,-0.016133,-0.004362,0.001465,1.000000,0.026464,0.009264,-0.124068,-0.043547,0.030873,-0.076749


In [26]:
X[X['LifeSquare'] < X['Square']].corr()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1
Id,1.000000,0.013348,-0.004759,0.005104,0.005709,0.020947,0.011879,-0.002629,0.005708,0.015678,-0.011051,-0.010332,-0.006825,-0.005256,-0.004160,-0.008594
DistrictId,0.013348,1.000000,0.056823,-0.020529,-0.027271,0.011899,-0.084722,-0.134285,0.012861,0.019461,0.085469,0.008012,0.227016,0.267156,0.237917,0.144379
Rooms,-0.004759,0.056823,1.000000,0.691427,0.568310,-0.004432,0.005383,-0.029319,-0.012598,-0.030242,0.041340,0.040305,0.040175,0.027406,0.039981,0.058397
Square,0.005104,-0.020529,0.691427,1.000000,0.811406,0.019127,0.150891,0.170872,-0.010451,-0.057939,-0.056175,-0.019976,0.063194,-0.031134,-0.016002,0.047708
LifeSquare,0.005709,-0.027271,0.568310,0.811406,1.000000,0.012571,0.091907,0.097717,-0.009471,-0.050932,-0.089945,-0.062269,0.077410,-0.012345,-0.035715,0.032806
KitchenSquare,0.020947,0.011899,-0.004432,0.019127,0.012571,1.000000,0.001655,0.002588,0.000415,-0.013833,-0.000239,0.000321,-0.012495,-0.006789,0.018715,-0.002752
Floor,0.011879,-0.084722,0.005383,0.150891,0.091907,0.001655,1.000000,0.502211,0.002255,-0.020632,-0.006526,0.023162,-0.027114,-0.120143,-0.053354,0.035177
HouseFloor,-0.002629,-0.134285,-0.029319,0.170872,0.097717,0.002588,0.502211,1.000000,-0.000639,-0.025345,0.014277,0.050697,-0.057269,-0.147183,-0.062686,0.034401
HouseYear,0.005708,0.012861,-0.012598,-0.010451,-0.009471,0.000415,0.002255,-0.000639,1.000000,0.001210,0.000441,-0.000401,0.001763,-0.013638,0.011005,0.003743
Ecology_1,0.015678,0.019461,-0.030242,-0.057939,-0.050932,-0.013833,-0.020632,-0.025345,0.001210,1.000000,-0.077868,-0.087130,-0.068708,-0.011113,0.005456,-0.075156


In [27]:
X[X['LifeSquare'] < X['Square']].describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1
count,7405.000000,7405.000000,7405.000000,7405.000000,7405.000000,7405.000000,7405.000000,7405.000000,7.405000e+03,7405.000000,7405.000000,7405.000000,7405.000000,4288.000000,7405.000000,7405.000000
mean,8427.099392,58.217286,1.929912,55.006667,34.572353,7.831600,8.045240,12.355976,4.692835e+03,0.123756,29.370425,6273.028359,7.142741,1224.965252,1.564213,4.598244
std,4861.523720,45.824875,0.858163,19.693523,17.567566,32.935564,4.984821,6.056501,2.329981e+05,0.120515,16.272551,3852.741489,18.725772,1079.378366,1.514861,4.348503
min,0.000000,0.000000,0.000000,2.377248,0.370619,0.000000,1.000000,0.000000,1.910000e+03,0.000000,0.000000,168.000000,0.000000,30.000000,0.000000,0.000000
25%,4242.000000,22.000000,1.000000,41.207774,22.545442,5.000000,4.000000,9.000000,1.970000e+03,0.033494,21.000000,3893.000000,1.000000,325.000000,0.000000,1.000000
50%,8434.000000,47.000000,2.000000,50.476307,32.022982,8.000000,7.000000,12.000000,1.979000e+03,0.089040,30.000000,5731.000000,3.000000,1015.000000,1.000000,4.000000
75%,12671.000000,90.000000,3.000000,64.437930,43.778489,9.000000,11.000000,17.000000,2.003000e+03,0.194489,39.000000,7759.000000,5.000000,1904.750000,3.000000,6.000000
max,16798.000000,209.000000,19.000000,641.065193,638.163193,2014.000000,42.000000,99.000000,2.005201e+07,0.521867,74.000000,19083.000000,141.000000,4849.000000,6.000000,23.000000


In [30]:
X[X['LifeSquare'] < X['Square']]['LifeSquare'].median() / X[X['LifeSquare'] < X['Square']]['Square'].median()

0.6344161027220871

In [41]:
X.loc[X['LifeSquare'] > X['Square'], 'LifeSquare'] = X.loc[X['LifeSquare'] > X['Square'], 'Square'] * 0.6344161027220871

In [42]:
X.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1
count,10000.00000,10000.000000,10000.000000,10000.000000,7887.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,5202.000000,10000.000000,10000.000000
mean,8383.40770,50.400800,1.890500,56.315775,34.736589,6.273300,8.526700,12.609400,3.990166e+03,0.118858,24.687000,5352.157400,8.039200,1142.904460,1.319500,4.231300
std,4859.01902,43.587592,0.839512,21.058732,17.638497,28.560917,5.241148,6.775974,2.005003e+05,0.119025,17.532614,4006.799803,23.831875,1021.517264,1.493601,4.806341
min,0.00000,0.000000,0.000000,1.136859,0.370619,0.000000,1.000000,0.000000,1.910000e+03,0.000000,0.000000,168.000000,0.000000,0.000000,0.000000,0.000000
25%,4169.50000,20.000000,1.000000,41.774881,22.686697,1.000000,4.000000,9.000000,1.974000e+03,0.017647,6.000000,1564.000000,0.000000,350.000000,0.000000,1.000000
50%,8394.50000,36.000000,2.000000,52.513310,32.096343,6.000000,7.000000,13.000000,1.977000e+03,0.075424,25.000000,5285.000000,2.000000,900.000000,1.000000,3.000000
75%,12592.50000,75.000000,2.000000,65.900625,43.807982,9.000000,12.000000,17.000000,2.001000e+03,0.195781,36.000000,7227.000000,5.000000,1548.000000,2.000000,6.000000
max,16798.00000,209.000000,19.000000,641.065193,638.163193,2014.000000,42.000000,117.000000,2.005201e+07,0.521867,74.000000,19083.000000,141.000000,4849.000000,6.000000,23.000000


In [48]:
X['LifeSquare'] = X['LifeSquare'].fillna(X['Square'] * 0.6344161027220871)

In [51]:
X.corr()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1
Id,1.000000,0.012973,-0.005847,-0.010071,-0.006418,0.019880,0.001348,-0.008376,0.005004,0.018097,-0.000772,-0.002033,-0.009358,-0.003879,0.001502,-0.008718
DistrictId,0.012973,1.000000,0.071432,-0.026613,-0.033480,0.040358,-0.120373,-0.149051,0.013430,0.065294,0.246463,0.167479,0.136095,0.304335,0.306147,0.174214
Rooms,-0.005847,0.071432,1.000000,0.662893,0.577349,0.005123,-0.000665,-0.029302,-0.010612,-0.032347,0.075980,0.071335,0.012811,0.042857,0.063557,0.053618
Square,-0.010071,-0.026613,0.662893,1.000000,0.854276,0.008320,0.114791,0.081505,-0.009032,-0.064479,-0.070690,-0.043120,0.035241,-0.039748,-0.022960,0.021357
LifeSquare,-0.006418,-0.033480,0.577349,0.854276,1.000000,0.004832,0.079907,0.048064,-0.008916,-0.058503,-0.094375,-0.072147,0.047514,-0.024527,-0.039055,0.014853
KitchenSquare,0.019880,0.040358,0.005123,0.008320,0.004832,1.000000,-0.011397,0.000780,0.000958,-0.005622,0.043379,0.037805,-0.015140,0.009472,0.043170,0.010216
Floor,0.001348,-0.120373,-0.000665,0.114791,0.079907,-0.011397,1.000000,0.418986,0.000928,-0.016133,-0.044914,-0.016560,-0.002237,-0.138294,-0.065537,0.024264
HouseFloor,-0.008376,-0.149051,-0.029302,0.081505,0.048064,0.000780,0.418986,1.000000,-0.000864,-0.004362,-0.020801,0.007194,-0.008137,-0.143973,-0.068728,0.026279
HouseYear,0.005004,0.013430,-0.010612,-0.009032,-0.008916,0.000958,0.000928,-0.000864,1.000000,0.001465,0.003026,0.001970,0.000819,-0.011969,0.011245,0.003681
Ecology_1,0.018097,0.065294,-0.032347,-0.064479,-0.058503,-0.005622,-0.016133,-0.004362,0.001465,1.000000,0.026464,0.009264,-0.124068,-0.043547,0.030873,-0.076749


In [52]:
X.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1
count,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,5202.000000,10000.000000,10000.000000
mean,8383.40770,50.400800,1.890500,56.315775,35.487148,6.273300,8.526700,12.609400,3.990166e+03,0.118858,24.687000,5352.157400,8.039200,1142.904460,1.319500,4.231300
std,4859.01902,43.587592,0.839512,21.058732,17.083559,28.560917,5.241148,6.775974,2.005003e+05,0.119025,17.532614,4006.799803,23.831875,1021.517264,1.493601,4.806341
min,0.00000,0.000000,0.000000,1.136859,0.370619,0.000000,1.000000,0.000000,1.910000e+03,0.000000,0.000000,168.000000,0.000000,0.000000,0.000000,0.000000
25%,4169.50000,20.000000,1.000000,41.774881,24.197429,1.000000,4.000000,9.000000,1.974000e+03,0.017647,6.000000,1564.000000,0.000000,350.000000,0.000000,1.000000
50%,8394.50000,36.000000,2.000000,52.513310,32.854536,6.000000,7.000000,13.000000,1.977000e+03,0.075424,25.000000,5285.000000,2.000000,900.000000,1.000000,3.000000
75%,12592.50000,75.000000,2.000000,65.900625,43.786672,9.000000,12.000000,17.000000,2.001000e+03,0.195781,36.000000,7227.000000,5.000000,1548.000000,2.000000,6.000000
max,16798.00000,209.000000,19.000000,641.065193,638.163193,2014.000000,42.000000,117.000000,2.005201e+07,0.521867,74.000000,19083.000000,141.000000,4849.000000,6.000000,23.000000


In [53]:
X[['DistrictId', 'Social_3', 'Helthcare_2', 'Healthcare_1']]

,DistrictId,Social_3,Helthcare_2
0,35,5,0
1,41,1,1
2,53,0,1
3,58,3,0
4,99,1,2
...,...,...,...
9995,32,6,3
9996,18,0,0
9997,27,1,0
9998,75,9,2


In [55]:
X_train_hc1 = X.loc[X['Healthcare_1'].notnull(), ['DistrictId', 'Social_3', 'Helthcare_2', 'Healthcare_1']]
X_train_hc1

,DistrictId,Social_3,Helthcare_2,Healthcare_1
1,41,1,1,240.0
2,53,0,1,229.0
3,58,3,0,1084.0
4,99,1,2,2078.0
5,59,4,0,990.0
...,...,...,...,...
9990,1,1,1,200.0
9991,3,0,3,1937.0
9995,32,6,3,350.0
9998,75,9,2,325.0


In [58]:
X_test_hc1 = X.loc[X['Healthcare_1'].isnull(), ['DistrictId', 'Social_3', 'Helthcare_2']]
X_test_hc1

,DistrictId,Social_3,Helthcare_2
0,35,5,0
6,154,14,1
7,74,3,0
8,1,0,0
9,23,3,0
...,...,...,...
9992,150,6,1
9993,13,2,5
9994,23,3,0
9996,18,0,0


In [59]:
y_train_hc1 = X_train_hc1['Healthcare_1']
y_train_hc1

1        240.0
2        229.0
3       1084.0
4       2078.0
5        990.0
         ...  
9990     200.0
9991    1937.0
9995     350.0
9998     325.0
9999      30.0
Name: Healthcare_1, Length: 5202, dtype: float64

In [65]:
X.iloc[X_test_hc1.index, :]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2
0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969,0.089040,B,B,33,7976,5,NaN,0,11,B
6,14549,154,2.0,62.254114,37.160377,7.0,3,5.0,1960,0.460556,B,B,20,4386,14,NaN,1,5,B
7,11993,74,2.0,80.312926,50.951813,0.0,14,0.0,1977,0.075779,B,B,6,1437,3,NaN,0,2,B
8,5172,1,2.0,64.511437,40.927095,1.0,9,17.0,1977,0.007122,B,B,1,264,0,NaN,0,1,B
9,8649,23,1.0,46.461409,18.915552,8.0,13,17.0,2014,0.075779,B,B,6,1437,3,NaN,0,2,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9992,14333,150,3.0,78.249637,49.385096,10.0,5,16.0,1976,0.300323,B,B,52,10311,6,NaN,1,9,B
9993,48,13,3.0,80.631333,48.899083,10.0,11,14.0,1999,0.090799,B,B,74,19083,2,NaN,5,15,B
9994,4158,23,2.0,64.864198,41.150892,1.0,17,17.0,1977,0.075779,B,B,6,1437,3,NaN,0,2,B
9996,6159,18,1.0,41.521546,20.539216,9.0,13,13.0,2000,0.000000,B,B,30,5562,0,NaN,0,5,A


In [66]:
X_train_hc1.drop('Healthcare_1', axis=1, inplace=True)
X_train_hc1

,DistrictId,Social_3,Helthcare_2
1,41,1,1
2,53,0,1
3,58,3,0
4,99,1,2
5,59,4,0
...,...,...,...
9990,1,1,1
9991,3,0,3
9995,32,6,3
9998,75,9,2


In [67]:
from sklearn.linear_model import LinearRegression

In [68]:
lr = LinearRegression()

In [69]:
lr.fit(X_train_hc1, y_train_hc1)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [70]:
y_pred_hc1 = lr.predict(X_test_hc1)
y_pred_hc1

array([ 853.12497218, 1635.29054335, 1072.56925672, ...,  778.23362026,
        740.92332324,  795.68286017])

In [71]:
X_test_hc1['y_pred_hc1'] = y_pred_hc1
X_test_hc1

,DistrictId,Social_3,Helthcare_2,y_pred_hc1
0,35,5,0,853.124972
6,154,14,1,1635.290543
7,74,3,0,1072.569257
8,1,0,0,642.811444
9,23,3,0,778.233620
...,...,...,...,...
9992,150,6,1,1589.661763
9993,13,2,5,1067.806962
9994,23,3,0,778.233620
9996,18,0,0,740.923323


In [74]:
X.iloc[X_train_hc1.index, :].describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1
count,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5.202000e+03,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000,5202.000000
mean,8380.733372,58.164360,1.922914,55.694241,34.867484,6.657824,8.212226,12.405229,5.837143e+03,0.145402,28.277201,5918.926759,13.179354,1142.904460,1.900038,5.081892
std,4854.277437,47.304299,0.818937,22.228970,17.810481,27.601105,5.111028,6.579024,2.779904e+05,0.128172,13.174587,2700.135491,31.935192,1021.517264,1.516684,5.262228
min,0.000000,0.000000,0.000000,2.596351,0.370619,0.000000,1.000000,0.000000,1.912000e+03,0.000000,2.000000,528.000000,0.000000,0.000000,0.000000,0.000000
25%,4202.500000,17.000000,1.000000,41.466277,23.716329,5.000000,4.000000,9.000000,1.969000e+03,0.035145,22.000000,4378.000000,1.000000,350.000000,1.000000,2.000000
50%,8380.500000,52.000000,2.000000,51.390424,32.292002,6.000000,7.000000,12.000000,1.977000e+03,0.130618,27.000000,5735.000000,2.000000,900.000000,2.000000,4.000000
75%,12593.500000,90.000000,2.000000,65.270484,43.190914,9.000000,11.000000,17.000000,1.996000e+03,0.243205,36.000000,7287.000000,8.000000,1548.000000,3.000000,6.000000
max,16798.000000,209.000000,10.000000,641.065193,638.163193,1970.000000,42.000000,117.000000,2.005201e+07,0.458174,66.000000,12238.000000,141.000000,4849.000000,6.000000,23.000000


In [73]:
X_test_hc1.describe()

,DistrictId,Social_3,Helthcare_2,y_pred_hc1
count,4798.000000,4798.000000,4798.000000,4798.000000
mean,41.983535,2.466236,0.690079,934.608846
std,37.386871,4.289941,1.180778,252.616298
min,1.000000,0.000000,0.000000,642.811444
25%,23.000000,0.000000,0.000000,770.545148
50%,27.000000,1.000000,0.000000,813.626732
75%,62.000000,3.000000,1.000000,1072.569257
max,208.000000,45.000000,5.000000,2035.827394


In [91]:
#Столбец 15 соответствует Healthcare_1
X.iloc[X_test_hc1.index, 15]

0      NaN
6      NaN
7      NaN
8      NaN
9      NaN
        ..
9992   NaN
9993   NaN
9994   NaN
9996   NaN
9997   NaN
Name: Healthcare_1, Length: 4798, dtype: float64

In [94]:
X_test_hc1['y_pred_hc1']

0        853.124972
6       1635.290543
7       1072.569257
8        642.811444
9        778.233620
           ...     
9992    1589.661763
9993    1067.806962
9994     778.233620
9996     740.923323
9997     795.682860
Name: y_pred_hc1, Length: 4798, dtype: float64

In [93]:
# заменяем пустые значения Healthcare_1 предсказанными значениями по 'DistrictId', 'Social_3', 'Helthcare_2' (признаки с наибольшей корреляцией)
X.iloc[X_test_hc1.index, 15] = X_test_hc1['y_pred_hc1']

In [95]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             10000 non-null  int64  
 1   DistrictId     10000 non-null  int64  
 2   Rooms          10000 non-null  float64
 3   Square         10000 non-null  float64
 4   LifeSquare     10000 non-null  float64
 5   KitchenSquare  10000 non-null  float64
 6   Floor          10000 non-null  int64  
 7   HouseFloor     10000 non-null  float64
 8   HouseYear      10000 non-null  int64  
 9   Ecology_1      10000 non-null  float64
 10  Ecology_2      10000 non-null  object 
 11  Ecology_3      10000 non-null  object 
 12  Social_1       10000 non-null  int64  
 13  Social_2       10000 non-null  int64  
 14  Social_3       10000 non-null  int64  
 15  Healthcare_1   10000 non-null  float64
 16  Helthcare_2    10000 non-null  int64  
 17  Shops_1        10000 non-null  int64  
 18  Shops_2

In [96]:
X.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1
count,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,8383.40770,50.400800,1.890500,56.315775,35.487148,6.273300,8.526700,12.609400,3.990166e+03,0.118858,24.687000,5352.157400,8.039200,1042.964224,1.319500,4.231300
std,4859.01902,43.587592,0.839512,21.058732,17.083559,28.560917,5.241148,6.775974,2.005003e+05,0.119025,17.532614,4006.799803,23.831875,764.344554,1.493601,4.806341
min,0.00000,0.000000,0.000000,1.136859,0.370619,0.000000,1.000000,0.000000,1.910000e+03,0.000000,0.000000,168.000000,0.000000,0.000000,0.000000,0.000000
25%,4169.50000,20.000000,1.000000,41.774881,24.197429,1.000000,4.000000,9.000000,1.974000e+03,0.017647,6.000000,1564.000000,0.000000,642.811444,0.000000,1.000000
50%,8394.50000,36.000000,2.000000,52.513310,32.854536,6.000000,7.000000,13.000000,1.977000e+03,0.075424,25.000000,5285.000000,2.000000,862.614981,1.000000,3.000000
75%,12592.50000,75.000000,2.000000,65.900625,43.786672,9.000000,12.000000,17.000000,2.001000e+03,0.195781,36.000000,7227.000000,5.000000,1173.769847,2.000000,6.000000
max,16798.00000,209.000000,19.000000,641.065193,638.163193,2014.000000,42.000000,117.000000,2.005201e+07,0.521867,74.000000,19083.000000,141.000000,4849.000000,6.000000,23.000000


In [108]:
X['HouseYear'].unique()

array([1969., 1978., 1968., 1977., 1976., 2011., 1960., 2014., 1973.,
       1959., 1999., 1980., 1979., 1983., 2001., 2012., 2002., 1996.,
       1964., 2018., 1972., 1965., 1984., 1961., 1971., 1963., 2017.,
       1970., 1981., 2003., 2016., 1991., 1975., 2006., 2009., 1985.,
       1974., 1994., 2000., 1987., 1998., 2005., 1990., 1982., 1997.,
       2015., 2008., 2010., 2004., 2007., 1967., 1957., 1962., 1993.,
       1966., 1955., 1937., 1992., 1954., 1995., 2019., 1948., 1986.,
       2013., 1989., 1958., 1938., 1956., 1988., 2020., 1951., 1952.,
       1935., 1914., 1932., 1950., 1917., 1918., 1940., 1942., 1939.,
       1934., 1931., 1919., 1912., 1953., 1936., 1947., 1929., 1930.,
       1933., 1941., 1916., 1910., 1928.])

In [103]:
X.loc[X['HouseYear'] == 20052011, 'HouseYear'] = 2005

In [106]:
X[X['HouseYear'] > 2020]['HouseYear']

4189    4968
Name: HouseYear, dtype: int64

In [107]:
X.loc[X['HouseYear'] > 2020, 'HouseYear'] = X['HouseYear'].median()

In [121]:
X[X['KitchenSquare'] < X['LifeSquare']]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2
0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969.0,0.089040,B,B,33,7976,5,853.124972,0,11,B
1,15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978.0,0.000070,B,B,46,10309,1,240.000000,1,16,B
2,4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968.0,0.049637,B,B,34,7759,0,229.000000,1,3,B
3,5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977.0,0.437885,B,B,23,5735,3,1084.000000,0,5,B
4,10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976.0,0.012339,B,B,35,5776,1,2078.000000,2,4,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,77,32,2.0,50.401785,30.476203,5.0,6,5.0,1968.0,0.135650,B,B,46,7960,6,350.000000,3,11,B
9996,6159,18,1.0,41.521546,20.539216,9.0,13,13.0,2000.0,0.000000,B,B,30,5562,0,740.923323,0,5,A
9997,5123,27,1.0,47.939008,30.413278,1.0,12,16.0,2015.0,0.072158,B,B,2,629,1,795.682860,0,0,A
9998,5400,75,2.0,43.602562,33.840147,8.0,1,5.0,1961.0,0.307467,B,A,30,5048,9,325.000000,2,5,B


In [120]:
X[X['KitchenSquare'] < X['Square'] * 0.6344161027220871]['KitchenSquare'].median() / X[X['KitchenSquare'] < X['Square'] * 0.6344161027220871]['Square'].median()

0.11425674764515831

In [122]:
X.loc[X['KitchenSquare'] > X['Square'] * 0.6344161027220871, 'KitchenSquare'] = X.loc[X['KitchenSquare'] > X['Square'] * 0.6344161027220871, 'Square'] * 0.11425674764515831

In [124]:
X.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1
count,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,8383.40770,50.400800,1.890500,56.315775,35.487148,5.716366,8.526700,12.609400,1984.866600,0.118858,24.687000,5352.157400,8.039200,1042.964224,1.319500,4.231300
std,4859.01902,43.587592,0.839512,21.058732,17.083559,3.997735,5.241148,6.775974,18.410913,0.119025,17.532614,4006.799803,23.831875,764.344554,1.493601,4.806341
min,0.00000,0.000000,0.000000,1.136859,0.370619,0.000000,1.000000,0.000000,1910.000000,0.000000,0.000000,168.000000,0.000000,0.000000,0.000000,0.000000
25%,4169.50000,20.000000,1.000000,41.774881,24.197429,1.000000,4.000000,9.000000,1974.000000,0.017647,6.000000,1564.000000,0.000000,642.811444,0.000000,1.000000
50%,8394.50000,36.000000,2.000000,52.513310,32.854536,6.000000,7.000000,13.000000,1977.000000,0.075424,25.000000,5285.000000,2.000000,862.614981,1.000000,3.000000
75%,12592.50000,75.000000,2.000000,65.900625,43.786672,9.000000,12.000000,17.000000,2001.000000,0.195781,36.000000,7227.000000,5.000000,1173.769847,2.000000,6.000000
max,16798.00000,209.000000,19.000000,641.065193,638.163193,43.000000,42.000000,117.000000,2020.000000,0.521867,74.000000,19083.000000,141.000000,4849.000000,6.000000,23.000000


In [125]:
X['Rooms'].unique()

array([ 2.,  3.,  1.,  4., 10.,  0.,  5., 19.,  6.])

In [126]:
X['Rooms'].value_counts()

2.0     3880
1.0     3705
3.0     2235
4.0      150
5.0       18
0.0        8
10.0       2
19.0       1
6.0        1
Name: Rooms, dtype: int64

In [127]:
X[X['Rooms'] == 19]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2
1454,8491,1,19.0,42.006046,21.779288,7.0,17,17.0,2014.0,0.007122,B,B,1,264,0,642.811444,0,1,B


In [137]:
X[(X['Square'] > 41) & (X['Square'] < 43)]['Rooms'].value_counts()

1.0     617
2.0     129
5.0       1
0.0       1
19.0      1
Name: Rooms, dtype: int64

In [133]:
X[X['Rooms'] == 10]

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2
377,5927,57,10.0,59.056975,36.223072,10.0,22,22.0,2002.0,0.090799,B,B,74,19083,2,1321.74359,5,15,B
8849,14865,9,10.0,60.871266,38.420681,10.0,3,2.0,1994.0,0.161532,B,B,25,5648,1,30.00000,2,4,B


In [135]:
X[(X['Square'] > 59) & (X['Square'] < 61)]['Rooms'].value_counts()

2.0     230
3.0      95
1.0      17
10.0      2
6.0       1
Name: Rooms, dtype: int64

In [138]:
X.loc[X['Rooms'] == 19, 'Rooms'] = 1
X.loc[X['Rooms'] == 10, 'Rooms'] = 2

In [139]:
X.describe()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1
count,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,8383.40770,50.400800,1.887100,56.315775,35.487148,5.716366,8.526700,12.609400,1984.866600,0.118858,24.687000,5352.157400,8.039200,1042.964224,1.319500,4.231300
std,4859.01902,43.587592,0.813892,21.058732,17.083559,3.997735,5.241148,6.775974,18.410913,0.119025,17.532614,4006.799803,23.831875,764.344554,1.493601,4.806341
min,0.00000,0.000000,0.000000,1.136859,0.370619,0.000000,1.000000,0.000000,1910.000000,0.000000,0.000000,168.000000,0.000000,0.000000,0.000000,0.000000
25%,4169.50000,20.000000,1.000000,41.774881,24.197429,1.000000,4.000000,9.000000,1974.000000,0.017647,6.000000,1564.000000,0.000000,642.811444,0.000000,1.000000
50%,8394.50000,36.000000,2.000000,52.513310,32.854536,6.000000,7.000000,13.000000,1977.000000,0.075424,25.000000,5285.000000,2.000000,862.614981,1.000000,3.000000
75%,12592.50000,75.000000,2.000000,65.900625,43.786672,9.000000,12.000000,17.000000,2001.000000,0.195781,36.000000,7227.000000,5.000000,1173.769847,2.000000,6.000000
max,16798.00000,209.000000,6.000000,641.065193,638.163193,43.000000,42.000000,117.000000,2020.000000,0.521867,74.000000,19083.000000,141.000000,4849.000000,6.000000,23.000000


In [144]:
X

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2
0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969.0,0.089040,B,B,33,7976,5,853.124972,0,11,B
1,15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978.0,0.000070,B,B,46,10309,1,240.000000,1,16,B
2,4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968.0,0.049637,B,B,34,7759,0,229.000000,1,3,B
3,5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977.0,0.437885,B,B,23,5735,3,1084.000000,0,5,B
4,10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976.0,0.012339,B,B,35,5776,1,2078.000000,2,4,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,77,32,2.0,50.401785,30.476203,5.0,6,5.0,1968.0,0.135650,B,B,46,7960,6,350.000000,3,11,B
9996,6159,18,1.0,41.521546,20.539216,9.0,13,13.0,2000.0,0.000000,B,B,30,5562,0,740.923323,0,5,A
9997,5123,27,1.0,47.939008,30.413278,1.0,12,16.0,2015.0,0.072158,B,B,2,629,1,795.682860,0,0,A
9998,5400,75,2.0,43.602562,33.840147,8.0,1,5.0,1961.0,0.307467,B,A,30,5048,9,325.000000,2,5,B


In [154]:
X['Ecology_2'] = X['Ecology_2'].astype('category')

In [155]:
X['Ecology_3'] = X['Ecology_3'].astype('category')

In [156]:
X['Shops_2'] = X['Shops_2'].astype('category')

In [159]:
X = pd.get_dummies(X)

In [162]:
X

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,...,Social_3,Healthcare_1,Helthcare_2,Shops_1,Ecology_2_A,Ecology_2_B,Ecology_3_A,Ecology_3_B,Shops_2_A,Shops_2_B
0,14038,35,2.0,47.981561,29.442751,6.0,7,9.0,1969.0,0.089040,...,5,853.124972,0,11,0,1,0,1,0,1
1,15053,41,3.0,65.683640,40.049543,8.0,7,9.0,1978.0,0.000070,...,1,240.000000,1,16,0,1,0,1,0,1
2,4765,53,2.0,44.947953,29.197612,0.0,8,12.0,1968.0,0.049637,...,0,229.000000,1,3,0,1,0,1,0,1
3,5809,58,2.0,53.352981,52.731512,9.0,8,17.0,1977.0,0.437885,...,3,1084.000000,0,5,0,1,0,1,0,1
4,10783,99,1.0,39.649192,23.776169,7.0,11,12.0,1976.0,0.012339,...,1,2078.000000,2,4,0,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,77,32,2.0,50.401785,30.476203,5.0,6,5.0,1968.0,0.135650,...,6,350.000000,3,11,0,1,0,1,0,1
9996,6159,18,1.0,41.521546,20.539216,9.0,13,13.0,2000.0,0.000000,...,0,740.923323,0,5,0,1,0,1,1,0
9997,5123,27,1.0,47.939008,30.413278,1.0,12,16.0,2015.0,0.072158,...,1,795.682860,0,0,0,1,0,1,1,0
9998,5400,75,2.0,43.602562,33.840147,8.0,1,5.0,1961.0,0.307467,...,9,325.000000,2,5,0,1,1,0,0,1


In [163]:
X.drop('Id', axis = 1, inplace = True)
X

,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,...,Social_3,Healthcare_1,Helthcare_2,Shops_1,Ecology_2_A,Ecology_2_B,Ecology_3_A,Ecology_3_B,Shops_2_A,Shops_2_B
0,35,2.0,47.981561,29.442751,6.0,7,9.0,1969.0,0.089040,33,...,5,853.124972,0,11,0,1,0,1,0,1
1,41,3.0,65.683640,40.049543,8.0,7,9.0,1978.0,0.000070,46,...,1,240.000000,1,16,0,1,0,1,0,1
2,53,2.0,44.947953,29.197612,0.0,8,12.0,1968.0,0.049637,34,...,0,229.000000,1,3,0,1,0,1,0,1
3,58,2.0,53.352981,52.731512,9.0,8,17.0,1977.0,0.437885,23,...,3,1084.000000,0,5,0,1,0,1,0,1
4,99,1.0,39.649192,23.776169,7.0,11,12.0,1976.0,0.012339,35,...,1,2078.000000,2,4,0,1,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,32,2.0,50.401785,30.476203,5.0,6,5.0,1968.0,0.135650,46,...,6,350.000000,3,11,0,1,0,1,0,1
9996,18,1.0,41.521546,20.539216,9.0,13,13.0,2000.0,0.000000,30,...,0,740.923323,0,5,0,1,0,1,1,0
9997,27,1.0,47.939008,30.413278,1.0,12,16.0,2015.0,0.072158,2,...,1,795.682860,0,0,0,1,0,1,1,0
9998,75,2.0,43.602562,33.840147,8.0,1,5.0,1961.0,0.307467,30,...,9,325.000000,2,5,0,1,1,0,0,1


In [164]:
from sklearn.model_selection import train_test_split

In [165]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [166]:
model = RandomForestRegressor(n_estimators=1000, max_depth=14, random_state=42)

In [171]:
y_train.values

array([210849.69354198, 201549.99807748, 158433.16877471, ...,
       255102.40670871, 182236.04875882, 443365.49645106])

In [172]:
model.fit(X_train, y_train.values)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=14, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=1000, n_jobs=None, oob_score=False,
                      random_state=42, verbose=0, warm_start=False)

In [175]:
y_pred = model.predict(X_test)

In [176]:
r2_score(y_test, y_pred)

0.7188278199694464

In [227]:
Z = pd.read_csv('test.csv')

In [228]:
Z.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             5000 non-null   int64  
 1   DistrictId     5000 non-null   int64  
 2   Rooms          5000 non-null   float64
 3   Square         5000 non-null   float64
 4   LifeSquare     3959 non-null   float64
 5   KitchenSquare  5000 non-null   float64
 6   Floor          5000 non-null   int64  
 7   HouseFloor     5000 non-null   float64
 8   HouseYear      5000 non-null   int64  
 9   Ecology_1      5000 non-null   float64
 10  Ecology_2      5000 non-null   object 
 11  Ecology_3      5000 non-null   object 
 12  Social_1       5000 non-null   int64  
 13  Social_2       5000 non-null   int64  
 14  Social_3       5000 non-null   int64  
 15  Healthcare_1   2623 non-null   float64
 16  Helthcare_2    5000 non-null   int64  
 17  Shops_1        5000 non-null   int64  
 18  Shops_2 

In [229]:
Z.corr()

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1
Id,1.000000,-0.018301,-0.006171,0.003529,0.038336,0.004892,-0.001000,0.008205,-0.002269,-0.002440,-0.025933,-0.021284,0.007603,-0.023668,-0.022600,-0.010774
DistrictId,-0.018301,1.000000,0.048620,-0.037734,-0.063092,0.075354,-0.130514,-0.151899,-0.210673,0.080369,0.221359,0.145558,0.131288,0.314152,0.277480,0.157297
Rooms,-0.006171,0.048620,1.000000,0.737178,0.560738,0.055058,-0.022866,-0.008065,-0.022069,-0.012659,0.070414,0.068312,-0.006876,0.026028,0.028808,0.036813
Square,0.003529,-0.037734,0.737178,1.000000,0.710879,0.043994,0.112994,0.134888,0.221221,-0.055763,-0.086439,-0.052063,0.003502,-0.062007,-0.068009,-0.010220
LifeSquare,0.038336,-0.063092,0.560738,0.710879,1.000000,0.006437,0.087459,0.120507,0.206685,-0.055910,-0.127617,-0.097183,0.033452,-0.052566,-0.071425,-0.014497
KitchenSquare,0.004892,0.075354,0.055058,0.043994,0.006437,1.000000,-0.011754,0.072769,0.074535,0.004132,0.131234,0.120183,-0.037653,0.021674,0.077121,0.029707
Floor,-0.001000,-0.130514,-0.022866,0.112994,0.087459,-0.011754,1.000000,0.463441,0.280718,-0.027548,-0.041018,-0.005598,0.007108,-0.119675,-0.048595,0.042854
HouseFloor,0.008205,-0.151899,-0.008065,0.134888,0.120507,0.072769,0.463441,1.000000,0.418111,-0.036547,-0.035906,-0.002948,-0.027500,-0.161227,-0.079672,0.017493
HouseYear,-0.002269,-0.210673,-0.022069,0.221221,0.206685,0.074535,0.280718,0.418111,1.000000,-0.074931,-0.057470,0.006975,-0.074811,-0.247141,-0.130654,-0.017997
Ecology_1,-0.002440,0.080369,-0.012659,-0.055763,-0.055910,0.004132,-0.027548,-0.036547,-0.074931,1.000000,0.016759,0.001057,-0.120340,-0.023860,0.024136,-0.075825


In [231]:
Z['LifeSquare'] = Z['LifeSquare'].fillna(Z['Square'] * 0.6344161027220871)

In [232]:
Z_test_hc1 = Z.loc[Z['Healthcare_1'].isnull(), ['DistrictId', 'Social_3', 'Helthcare_2']]
Z_test_hc1

,DistrictId,Social_3,Helthcare_2
0,58,1,0
1,74,3,0
3,47,3,3
4,27,1,0
6,23,3,0
...,...,...,...
4992,96,4,1
4995,11,0,1
4996,1,0,0
4997,12,2,5


In [233]:
res_pred_hc1 = lr.predict(Z_test_hc1)
res_pred_hc1

array([ 974.59275684, 1072.56925672, 1126.80700761, ...,  642.81144442,
       1062.03567535,  997.67790479])

In [234]:
Z_test_hc1['res_pred_hc1'] = res_pred_hc1
Z_test_hc1

,DistrictId,Social_3,Helthcare_2,res_pred_hc1
0,58,1,0,974.592757
1,74,3,0,1072.569257
3,47,3,3,1126.807008
4,27,1,0,795.682860
6,23,3,0,778.233620
...,...,...,...,...
4992,96,4,1,1272.376358
4995,11,0,1,770.545148
4996,1,0,0,642.811444
4997,12,2,5,1062.035675


In [235]:
# пустые значения Healthcare_1 тестовой выборки
Z.iloc[Z_test_hc1.index, 15]

0      NaN
1      NaN
3      NaN
4      NaN
6      NaN
        ..
4992   NaN
4995   NaN
4996   NaN
4997   NaN
4998   NaN
Name: Healthcare_1, Length: 2377, dtype: float64

In [236]:
# заменяем пустые значения Healthcare_1 предсказанными значениями по 'DistrictId', 'Social_3', 'Helthcare_2' (признаки с наибольшей корреляцией)
Z.iloc[Z_test_hc1.index, 15] = Z_test_hc1['res_pred_hc1']

In [237]:
Z.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 19 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             5000 non-null   int64  
 1   DistrictId     5000 non-null   int64  
 2   Rooms          5000 non-null   float64
 3   Square         5000 non-null   float64
 4   LifeSquare     5000 non-null   float64
 5   KitchenSquare  5000 non-null   float64
 6   Floor          5000 non-null   int64  
 7   HouseFloor     5000 non-null   float64
 8   HouseYear      5000 non-null   int64  
 9   Ecology_1      5000 non-null   float64
 10  Ecology_2      5000 non-null   object 
 11  Ecology_3      5000 non-null   object 
 12  Social_1       5000 non-null   int64  
 13  Social_2       5000 non-null   int64  
 14  Social_3       5000 non-null   int64  
 15  Healthcare_1   5000 non-null   float64
 16  Helthcare_2    5000 non-null   int64  
 17  Shops_1        5000 non-null   int64  
 18  Shops_2 

In [238]:
Z

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2
0,725,58,2.0,49.882643,33.432782,6.0,6,14.0,1972,0.310199,B,B,11,2748,1,974.592757,0,0,B
1,15856,74,2.0,69.263183,43.941679,1.0,6,1.0,1977,0.075779,B,B,6,1437,3,1072.569257,0,2,B
2,5480,190,1.0,13.597819,15.948246,12.0,2,5.0,1909,0.000000,B,B,30,7538,87,4702.000000,5,5,B
3,15664,47,2.0,73.046609,51.940842,9.0,22,22.0,2007,0.101872,B,B,23,4583,3,1126.807008,3,3,B
4,14275,27,1.0,47.527111,43.387569,1.0,17,17.0,2017,0.072158,B,B,2,629,1,795.682860,0,0,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,8180,11,3.0,67.133911,50.809797,6.0,5,9.0,1973,0.000170,B,B,36,5992,0,770.545148,1,1,B
4996,4695,1,1.0,40.198472,21.807061,10.0,12,17.0,2017,0.007122,B,B,1,264,0,642.811444,0,1,B
4997,5783,12,3.0,77.842178,48.282625,9.0,23,22.0,1989,0.090799,B,B,74,19083,2,1062.035675,5,15,B
4998,4780,62,2.0,81.305222,51.581342,0.0,4,0.0,1977,0.072158,B,B,2,629,1,997.677905,0,0,A


In [239]:
Z['Ecology_2'] = Z['Ecology_2'].astype('category')

In [240]:
Z['Ecology_3'] = Z['Ecology_3'].astype('category')

In [241]:
Z['Shops_2'] = Z['Shops_2'].astype('category')

In [242]:
Z = pd.get_dummies(Z)
Z

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,...,Social_3,Healthcare_1,Helthcare_2,Shops_1,Ecology_2_A,Ecology_2_B,Ecology_3_A,Ecology_3_B,Shops_2_A,Shops_2_B
0,725,58,2.0,49.882643,33.432782,6.0,6,14.0,1972,0.310199,...,1,974.592757,0,0,0,1,0,1,0,1
1,15856,74,2.0,69.263183,43.941679,1.0,6,1.0,1977,0.075779,...,3,1072.569257,0,2,0,1,0,1,0,1
2,5480,190,1.0,13.597819,15.948246,12.0,2,5.0,1909,0.000000,...,87,4702.000000,5,5,0,1,0,1,0,1
3,15664,47,2.0,73.046609,51.940842,9.0,22,22.0,2007,0.101872,...,3,1126.807008,3,3,0,1,0,1,0,1
4,14275,27,1.0,47.527111,43.387569,1.0,17,17.0,2017,0.072158,...,1,795.682860,0,0,0,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,8180,11,3.0,67.133911,50.809797,6.0,5,9.0,1973,0.000170,...,0,770.545148,1,1,0,1,0,1,0,1
4996,4695,1,1.0,40.198472,21.807061,10.0,12,17.0,2017,0.007122,...,0,642.811444,0,1,0,1,0,1,0,1
4997,5783,12,3.0,77.842178,48.282625,9.0,23,22.0,1989,0.090799,...,2,1062.035675,5,15,0,1,0,1,0,1
4998,4780,62,2.0,81.305222,51.581342,0.0,4,0.0,1977,0.072158,...,1,997.677905,0,0,0,1,0,1,1,0


In [243]:
Z.drop('Id', axis=1, inplace=True)
Z

,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Social_1,...,Social_3,Healthcare_1,Helthcare_2,Shops_1,Ecology_2_A,Ecology_2_B,Ecology_3_A,Ecology_3_B,Shops_2_A,Shops_2_B
0,58,2.0,49.882643,33.432782,6.0,6,14.0,1972,0.310199,11,...,1,974.592757,0,0,0,1,0,1,0,1
1,74,2.0,69.263183,43.941679,1.0,6,1.0,1977,0.075779,6,...,3,1072.569257,0,2,0,1,0,1,0,1
2,190,1.0,13.597819,15.948246,12.0,2,5.0,1909,0.000000,30,...,87,4702.000000,5,5,0,1,0,1,0,1
3,47,2.0,73.046609,51.940842,9.0,22,22.0,2007,0.101872,23,...,3,1126.807008,3,3,0,1,0,1,0,1
4,27,1.0,47.527111,43.387569,1.0,17,17.0,2017,0.072158,2,...,1,795.682860,0,0,0,1,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,11,3.0,67.133911,50.809797,6.0,5,9.0,1973,0.000170,36,...,0,770.545148,1,1,0,1,0,1,0,1
4996,1,1.0,40.198472,21.807061,10.0,12,17.0,2017,0.007122,1,...,0,642.811444,0,1,0,1,0,1,0,1
4997,12,3.0,77.842178,48.282625,9.0,23,22.0,1989,0.090799,74,...,2,1062.035675,5,15,0,1,0,1,0,1
4998,62,2.0,81.305222,51.581342,0.0,4,0.0,1977,0.072158,2,...,1,997.677905,0,0,0,1,0,1,1,0


In [244]:
# С помощью модели RandomForestRegressor, обученной на тренировочных данных, предскажем цены тестового датасета
res_pred = model.predict(Z)
res_pred

array([162580.41887162, 224985.7949969 , 198835.09669346, ...,
       329972.24955312, 205395.44115104, 182245.42842919])

In [245]:
T = pd.read_csv('test.csv')
T

,Id,DistrictId,Rooms,Square,LifeSquare,KitchenSquare,Floor,HouseFloor,HouseYear,Ecology_1,Ecology_2,Ecology_3,Social_1,Social_2,Social_3,Healthcare_1,Helthcare_2,Shops_1,Shops_2
0,725,58,2.0,49.882643,33.432782,6.0,6,14.0,1972,0.310199,B,B,11,2748,1,NaN,0,0,B
1,15856,74,2.0,69.263183,NaN,1.0,6,1.0,1977,0.075779,B,B,6,1437,3,NaN,0,2,B
2,5480,190,1.0,13.597819,15.948246,12.0,2,5.0,1909,0.000000,B,B,30,7538,87,4702.0,5,5,B
3,15664,47,2.0,73.046609,51.940842,9.0,22,22.0,2007,0.101872,B,B,23,4583,3,NaN,3,3,B
4,14275,27,1.0,47.527111,43.387569,1.0,17,17.0,2017,0.072158,B,B,2,629,1,NaN,0,0,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,8180,11,3.0,67.133911,50.809797,6.0,5,9.0,1973,0.000170,B,B,36,5992,0,NaN,1,1,B
4996,4695,1,1.0,40.198472,21.807061,10.0,12,17.0,2017,0.007122,B,B,1,264,0,NaN,0,1,B
4997,5783,12,3.0,77.842178,48.282625,9.0,23,22.0,1989,0.090799,B,B,74,19083,2,NaN,5,15,B
4998,4780,62,2.0,81.305222,NaN,0.0,4,0.0,1977,0.072158,B,B,2,629,1,NaN,0,0,A


In [256]:
final = pd.concat([T['Id'], pd.DataFrame(res_pred, columns=['Price'])], axis=1)
final

,Id,Price
0,725,162580.418872
1,15856,224985.794997
2,5480,198835.096693
3,15664,344529.470958
4,14275,145449.035391
...,...,...
4995,8180,253933.765907
4996,4695,126410.827403
4997,5783,329972.249553
4998,4780,205395.441151


In [258]:
final.to_csv('my_submission.csv', index=False)